In [2]:
#Importing libraries
import sys
import random
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA, TruncatedSVD

my_seed = 1337
random.seed(my_seed)
np.random.seed(my_seed)


In [3]:
data = pd.read_csv('rating_final.csv')

In [4]:
data.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [5]:
ratings = data[['rating','food_rating','service_rating']].copy()

In [6]:
ratings.head()

,rating,food_rating,service_rating
0,2,2,2
1,2,2,1
2,2,2,2
3,1,2,2
4,1,1,2


In [7]:
data = data.drop(data.columns[[3,4]], 1)

In [8]:
data.head()

,userID,placeID,rating
0,U1077,135085,2
1,U1077,135038,2
2,U1077,132825,2
3,U1077,135060,1
4,U1068,135104,1


In [9]:
frames = [data,ratings]

In [10]:
data.rating = ratings.rating.copy()

In [11]:
data.head()

,userID,placeID,rating
0,U1077,135085,2
1,U1077,135038,2
2,U1077,132825,2
3,U1077,135060,1
4,U1068,135104,1


In [20]:
userRatings = data.pivot_table(index=['userID'],columns=['placeID'],values='rating')
print("before: ",userRatings.shape)
userRatings = userRatings.dropna(thresh=5, axis=1).fillna(0,axis=1)

print("After: ",userRatings.shape)

After:  (138, 130)
After:  (138, 99)


In [21]:
userRatings.head()

placeID,132572,132584,132594,132608,132609,132613,132630,132660,132663,132665,...,135079,135080,135081,135082,135085,135086,135088,135104,135106,135108
userID,,,,,,,,,,,,,,,,,,,,,
U1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U1002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
U1003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U1004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
U1005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
corrMatrix = userRatings.corr(method='pearson')
corrMatrix.head(100)

placeID,132572,132584,132594,132608,132609,132613,132630,132660,132663,132665,...,135079,135080,135081,135082,135085,135086,135088,135104,135106,135108
placeID,,,,,,,,,,,,,,,,,,,,,
132572,1.000000,-0.053702,-0.043250,-0.047484,-0.043250,-0.052988,-0.058799,-0.048617,-0.043250,-0.040726,...,-0.007086,0.098133,-0.072199,-0.048617,0.077067,0.230333,-0.053267,-0.053267,0.133796,-0.021104
132584,-0.053702,1.000000,0.131066,0.056801,0.289724,0.216114,0.239814,0.580464,0.131066,-0.025983,...,-0.056507,-0.030294,-0.046062,-0.031017,-0.092190,-0.040819,-0.033983,0.454528,-0.041303,-0.046062
132594,-0.043250,0.131066,1.000000,0.162650,-0.022222,0.151687,0.168321,0.139175,0.659259,-0.020925,...,-0.045509,-0.024398,-0.037097,-0.024980,-0.074246,-0.032874,-0.027369,0.811948,-0.033264,-0.037097
132608,-0.047484,0.056801,0.162650,1.000000,0.536745,0.362961,0.075814,0.333022,-0.024398,-0.022974,...,-0.049963,-0.026786,-0.040728,-0.027425,-0.081514,-0.036092,-0.030048,0.200321,-0.036520,-0.040728
132609,-0.043250,0.289724,-0.022222,0.536745,1.000000,0.330599,0.168321,0.303330,-0.022222,-0.020925,...,-0.045509,-0.024398,-0.037097,-0.024980,-0.074246,-0.032874,-0.027369,0.182460,-0.033264,-0.037097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135086,0.230333,-0.040819,-0.032874,-0.036092,-0.032874,-0.040276,-0.044693,-0.036954,-0.032874,-0.030956,...,0.164941,0.171438,-0.054879,-0.036954,-0.070363,1.000000,-0.040488,-0.040488,-0.049209,-0.054879
135088,-0.053267,-0.033983,-0.027369,-0.030048,-0.027369,-0.033531,-0.037209,-0.030766,-0.027369,-0.025772,...,-0.056049,-0.030048,-0.045689,-0.030766,-0.091442,-0.040488,1.000000,-0.033708,-0.040968,-0.045689
135104,-0.053267,0.454528,0.811948,0.200321,0.182460,0.407168,0.207305,0.272496,0.602119,-0.025772,...,-0.056049,-0.030048,-0.045689,-0.030766,-0.091442,-0.040488,-0.033708,1.000000,-0.040968,-0.045689


In [46]:
def get_similar(placeID,rating):
    similar_ratings = corrMatrix[placeID]*(rating-1.5)
    similar_ratings = similar_ratings.sort_values(ascending=False)
    #print(type(similar_ratings))
    return similar_ratings

In [48]:
userx = [(135104,2),(132594,2),(132584,3),(132663,3)]
similar_movies = pd.DataFrame()
for movie,rating in userx:
    similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)

similar_movies.head(10)
similar_movies.sum().sort_values(ascending=False).head(20)

135104    2.490944
132663    2.327287
132594    2.091461
132584    1.989395
132733    1.583043
132660    1.285293
132732    1.235267
132613    1.099498
132630    1.097814
132740    0.742380
132609    0.481372
132608    0.230090
132870   -0.076746
132845   -0.093711
132858   -0.093711
132665   -0.093711
134983   -0.095683
135049   -0.095683
132885   -0.099518
132925   -0.108814
dtype: float64